# ipywidgets  深入浅出

ipywidgets 是一个在jupyter notebook 中开发用户交互界面的简单工具。

可以用它来做机器学习模型的演示，构建数据分析dashboard，或者做一些小工具。

相比streamlit和gradio，ipywidgets具有如下优势:

* 灵活高效：ipywidgets的组件可以和notebook的输出很好地结合在一起。

* 方便调试：ipywidgets和代码一起直接在jupyter中展示和运行， all in one notebook。

* 便于分享：任何可以托管notebook的环境都可以使用和展示它。

大多数的ipywidgets应用一般由如下最常用的基础模块构成。

* 应用界面：interact(简易场景), interact_manual(简易场景手动触发), interactive_output(定制化输入), display(完全定制化场景)

* 输入输出：Output(笔记本输出), Image(图像), Text(文本框), Textarea(文本块),  Dropdown(下拉选项), IntSlider(整数滑条), FloatSlider(浮点数滑条), HTML, Viedio(视频), 

* 控制组件：Button(按钮), Play(播放器) 

* 布局组件：Tab(标签页), HBox(行布局), VBox(列布局), Layout(外观)


我们将由易到难通过6个范例来介绍ipywidgets的使用方法。

* hello world范例

* 图片浏览工具

* 文本分类 

* 图片分类 

* 目标检测  

* 图片筛选器 

参考资料：

* 官方文档：https://ipywidgets.readthedocs.io/en/stable/

* 官方范例：https://github.com/jupyter-widgets/ipywidgets/tree/511663a56324cea5324f49a65ebe25e2f1b04d87/docs/source/examples





## 一，Hello World (难度系数: ⭐️)

In [ ]:
import ipywidgets as widgets
def greet(name):
    return "Hello " + name + "!!"

w = widgets.interact(greet,name='LiLei');
w


## 二，图片浏览工具 (难度系数: ⭐️⭐️)

In [ ]:
from ipywidgets import widgets,interact
from pathlib import Path
from PIL import Image
from  torchkeras.data import download_baidu_pictures 
download_baidu_pictures(keyword='猫咪表情包',needed_pics_num=20, save_dir = 'cats')
files = [str(x) for x in Path('cats').rglob('*.jpg') if 'checkpoint' not in str(x)]
def browser_image(path):
    return Image.open(path) 
    
interact(browser_image, path=files)


## 三，文本分类 (难度系数: ⭐️⭐️)

In [2]:
#解决国内下载huggingface仓库慢的问题
import os 
os.environ['HF_ENDPOINT']='https://hf-mirror.com'  

import ipywidgets as widgets
from transformers import pipeline

pipe = pipeline("text-classification")

@widgets.interact_manual(text='Good morning')
def clf(text):
    result = pipe(text)
    label = result[0]['label']
    score = result[0]['score']
    res = {label:score,'POSITIVE' if label=='NEGATIVE' else 'NEGATIVE': 1-score}
    return res 



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://hf-mirror.com/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


interactive(children=(Text(value='Good morning', continuous_update=False, description='text'), Button(descript…

## 四，图片分类 (难度系数: ⭐️⭐️⭐)

In [3]:
import ipywidgets as widgets
from IPython.display import HTML,display 


import pandas as pd 
from ultralytics import YOLO
from skimage import data
from PIL import Image

model = YOLO('yolov8n-cls.pt')
#prepare example image 
Image.fromarray(data.coffee()).save('coffee.jpeg') 
Image.fromarray(data.astronaut()).save('people.jpeg')
Image.fromarray(data.cat()).save('cat.jpeg')

def predict(img):
    result = model.predict(source=img)
    df = pd.Series(result[0].names).to_frame()
    df.columns = ['names']
    df['probs'] = result[0].probs.data.tolist()
    df = df.sort_values('probs',ascending=False).head(5)
    res = dict(zip(df['names'],df['probs']))
    return res

title = widgets.HTML("<h1 style='color:#e00b20;width:100%'> 图片分类展示系统</h1>")

files = widgets.Dropdown(
    options=['coffee.jpeg','people.jpeg','cat.jpeg'],
    value='people.jpeg',
    layout=widgets.Layout(width='100%', height='40px'),
    description='选择图片',
    disabled=False
)

button = widgets.Button(description='Predict',
                        layout=widgets.Layout(width='100%', height='40px'),
                        style=dict(font_weight='bold',text_color='red'),
                        button_style='primary'
                       )
button.style.button_color = 'lightgreen'

out1 = widgets.Output() #widgets.Textarea(value='',layout=widgets.Layout(width='100%',height='100px'))
out2 = widgets.Output()
output = widgets.HBox([out1,out2])

display(title,files,button,output)

def on_button_clicked(btn):
    import plotly.express as px 
    dic = predict(files.value)
    fig = px.bar(y = list(reversed(dic.keys())),x = list(reversed(dic.values())))
    
    out1.clear_output()
    out2.clear_output()
    with out1:
        display(Image.open(files.value))
    
    with out2:
        display(fig)

button.on_click(on_button_clicked)


HTML(value="<h1 style='color:#e00b20;width:100%'> 图片分类展示系统</h1>")

Dropdown(description='选择图片', index=1, layout=Layout(height='40px', width='100%'), options=('coffee.jpeg', 'peo…

Button(button_style='primary', description='Predict', layout=Layout(height='40px', width='100%'), style=Button…

## 五，目标检测 (难度系数: ⭐️⭐️⭐⭐️)

In [ ]:
import ipywidgets as widgets
import numpy as np 
import pandas as pd 
from skimage import data
from PIL import Image
from torchkeras import plots 
from torchkeras.data import get_url_img
from pathlib import Path
from ultralytics import YOLO
import ultralytics
from ultralytics.data import utils 
import cv2
import time

model = YOLO('yolov8n.pt')

#prepare example images
Image.fromarray(data.coffee()).save('coffee.jpeg') 
Image.fromarray(data.astronaut()).save('people.jpeg')
Image.fromarray(data.cat()).save('cat.jpeg')

#load class_names
yaml_path = str(Path(ultralytics.__file__).parent/'cfg/datasets/coco128.yaml') 
class_names = utils.yaml_load(yaml_path)['names']



def bgr2jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

def pil2jpeg(pil):
    import io
    buf = io.BytesIO()
    pil.save(buf,format='jpeg')
    return buf.getvalue()


def detect(img):
    if isinstance(img,str):
        img = get_url_img(img) if img.startswith('http') else Image.open(img).convert('RGB')
    result = model.predict(source=img)
    if result[0].boxes is not None and len(result[0].boxes.data.tolist())>0:
        vis = plots.plot_detection(img,boxes=result[0].boxes.data.tolist(),
                     class_names=class_names, min_score=0.2)
    else:
        vis = img
    return vis

def get_capture(height,width):
    cap = cv2.VideoCapture(0)  
    cap.set(cv2.CAP_PROP_FRAME_WIDTH,width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT,height)
    cap.set(3,width)       
    cap.set(4,height)
    cap.set(5, 30)  #设置帧率
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
    cap.set(cv2.CAP_PROP_BRIGHTNESS, 40) #设置亮度 -64 - 64  0.0
    cap.set(cv2.CAP_PROP_CONTRAST, 50)   #设置对比度 -64 - 64  2.0
    cap.set(cv2.CAP_PROP_EXPOSURE, 156)  #设置曝光值 1.0 - 5000  156.0
    return cap


In [ ]:
tabs = widgets.Tab(layout = widgets.Layout(width='100%', height='1000px'))

# TAB1: webcam
width,height=800,500

img1 = widgets.Image(format='jpeg', width=width, height=height,
                    layout=widgets.Layout(width='100%')) 
cap = get_capture(height,width)
ret, frame = cap.read()     
img1.value = bgr2jpeg(frame)

play = widgets.Play(
    value=0,
    min=0,
    max=100,
    step=1,
    interval=100,
    description="Press play",
    disabled=False
)

slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))

prompt = widgets.HTML('<p3>点击开始检测</p3>')

def on_value_change(change):
    ret, frame = cap.read()
    if change['new']<1:
         img1.value = bgr2jpeg(frame)
    else:
        out_img = detect(np.ascontiguousarray(frame[:,:,::-1]))
        img1.value = pil2jpeg(out_img)
play.observe(on_value_change, names='value')

tab1 = widgets.VBox([widgets.HBox([prompt,play,slider]),img1])
tabs.children = list(tabs.children)+[tab1]
tabs.set_title(0,'捕捉摄像头喔')


# TAB2: samples


samples = widgets.Dropdown(
    options=['coffee.jpeg','people.jpeg','cat.jpeg'],
    value='coffee.jpeg',
    layout=widgets.Layout(width='100%', height='40px'),
    description='',
    disabled=False
)

img2 = widgets.Image(format='jpeg', layout=widgets.Layout(width='100%'), value  = open('coffee.jpeg','rb').read()) 

def predict_sample(change):
    out_img = detect(change['new'])
    img2.value = pil2jpeg(out_img)
        
samples.observe(predict_sample, names='value')
tab2 = widgets.VBox([samples,img2])
tabs.children = list(tabs.children)+[tab2]
tabs.set_title(1,'选择测试图片')

# TAB3: url

default_url = 'https://t7.baidu.com/it/u=3601447414,1764260638&fm=193&f=GIF'
url = widgets.Text(value = default_url,layout=widgets.Layout(width='100%', height='30px'))
button = widgets.Button(description='执行预测',
                        layout=widgets.Layout(width='100%', height='40px'),
                        button_style='primary'
                       )
img3 = widgets.Image(format='jpeg', 
                     layout=widgets.Layout(width='100%')) 
button.style.button_color = 'brown'
def predict_url(btn):
    img3.value = pil2jpeg(detect(url.value))
        
button.on_click(predict_url)

tab3 = widgets.VBox([url,button,img3])
tabs.children = list(tabs.children)+[tab3]
tabs.set_title(2,'输入图片链接')

display(widgets.HTML('<h1>YOLOv8目标检测演示</h1>'),tabs)



## 六，图片筛选器 (难度系数: ⭐️⭐️⭐⭐️⭐️)



本范例我们将应用 ipywidgets来构建一个图片筛选器，从百度爬取的一堆猫咪表情包中刷选一些我们喜欢的出来。


In [ ]:
#!pip install -U torchkeras

In [ ]:
import torchkeras 
from importlib import reload 
reload(torchkeras)
from torchkeras.data import download_baidu_pictures 
download_baidu_pictures('猫咪表情包',100)


In [5]:
import ipywidgets as widgets 
from PIL import Image
import time,os
from pathlib import Path 
base_dir = 'cats'
selected_dir = 'selected'
files = [str(x) for x in 
         Path(base_dir).rglob('*.jp*g') 
         if 'checkpoint' not in str(x)]

def pil2jpeg(pil):
    import io
    buf = io.BytesIO()
    pil.save(buf,format='jpeg')
    return buf.getvalue()

def get_default_msg():
    if not os.path.exists(selected_dir):
        os.mkdir(selected_dir)
    selected_files = set(os.listdir(selected_dir))
    msg = 'Selected images number = {}\n'.format(len(selected_files))
    return msg 


done = widgets.IntText(description='已完成',value=0, layout=widgets.Layout(width='33%', height='auto'))
todo = widgets.IntText(description='待完成',value=len(files),layout=widgets.Layout(width='33%', height='auto'))
total = widgets.IntText(description='总数量',value = len(files), layout=widgets.Layout(width='33%', height='auto'))
int_group = widgets.HBox([total,done,todo])

path = widgets.Text(description='当前图片', value = files[0],layout=widgets.Layout(width='100%'))
        
bn_before = widgets.Button(description='上一张',layout=widgets.Layout(width='50%', height='auto'),button_style='info')
bn_next = widgets.Button(description='下一张',layout=widgets.Layout(width='50%', height='auto'),button_style='info')
bn_group = widgets.HBox([bn_before,bn_next],
                        style=widgets.Style(align='center'))

bn_feedback = widgets.Button(description='选择图片',
                             layout=widgets.Layout(width='100%', height='20pt'),button_style='primary')
bn_feedback.style.button_color = 'lightgreen'

info = widgets.Textarea(layout=widgets.Layout(width='100%', height='100pt'),value = get_default_msg())

img = widgets.Image(format='jpeg',value = pil2jpeg(Image.open(files[0])),
            layout=widgets.Layout(width='100%', height='auto'))


display(int_group,bn_group,bn_feedback,path,info,img)


def fn_before(btn):
    if done.value>=1:
        done.value = done.value-1
        todo.value= todo.value+1
    path.value = files[int(done.value-1)]
    img.value = pil2jpeg(Image.open(path.value))

def fn_next(btn):
    if todo.value>=1:
        done.value = done.value+1
        todo.value = todo.value-1   
    path.value = files[int(done.value-1)]
    img.value = pil2jpeg(Image.open(path.value))
    

def save_selected(btn):
    img_name = os.path.basename(path.value)
    img_path = path.value
    if img_path.startswith('http'):
        imgx = get_image(img_path).convert('RGB')
        img_path = 'tmp.jpg'
        imgx.save(img_path)
    selected_files = set(os.listdir(selected_dir))
    
    msg = ''
    if img_name not in selected_files:
        save_path = os.path.join(selected_dir,img_name)
        imgx = Image.open(img_path)
        imgx.save(save_path)
        msg = 'selected images number = {}\n'.format(len(selected_files)+1)+\
              'Save image sucessed!\n'+\
              'Saved image name : {}'.format(img_name)
    else:
        msg = 'selected images number = {}\n'.format(len(selected_files))+\
        "Don't save duplicate images!"
    info.value = msg

bn_before.on_click(fn_before)
bn_next.on_click(fn_next)
bn_feedback.on_click(save_selected)



Button(button_style='primary', description='选择图片', layout=Layout(height='20pt', width='100%'), style=ButtonSty…

Text(value='cats/20240624_052116_714619.jpg', description='当前图片', layout=Layout(width='100%'))

Textarea(value='Selected images number = 0\n', layout=Layout(height='100pt', width='100%'))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

## 七，隐藏代码

如果要把ipywidgets的交互页面给其他人使用，最好使用下面的方式来隐藏代码。

注，这个方法在jupyter lab中可能不太好使。

In [ ]:
def hide_jupyter_code():
    import ipywidgets as widgets
    from IPython.display import display, HTML
    javascript_functions = {False: "hide()", True: "show()"}
    button_descriptions  = {False: "Show code", True: "Hide code"}
    def toggle_code(state):
        output_string = "<script>$(\"div.input\").{}</script>"
        output_args   = (javascript_functions[state],)
        output        = output_string.format(*output_args)
        display(HTML(output))
    def button_action(value):
        state = value.new
        toggle_code(state)
        value.owner.description = button_descriptions[state]
    state = False
    toggle_code(state)
    button = widgets.ToggleButton(state, description = button_descriptions[state])
    button.observe(button_action, "value")
    display(button)

hide_jupyter_code()
